In [1]:
!pwd

/home/tungdao/tung/mlopsvn/code/mlops-crash-course-code/monitoring_service/nbs


In [2]:
import pandas as pd
import fastparquet
from pathlib import Path
import numpy as np

random_seed = 17
np.random.seed(random_seed)

## Load data

In [3]:
OUTSIDE_DATA_DIR = Path("../data")
ORIG_DATA_PATH = OUTSIDE_DATA_DIR / "orig_driver_stats.parquet"
NORMAL_DATA_PATH = OUTSIDE_DATA_DIR / "mock_normal_data.parquet"
DRIFT_DATA_PATH = OUTSIDE_DATA_DIR / "mock_drift_data.parquet"
REQUEST_DATA_PATH = OUTSIDE_DATA_DIR / "mock_request_data.csv"

In [46]:
def format_data_df(df, pred_prob):
    n_rows = df.shape[0]
    choices = np.random.choice([0, 1], size=n_rows, p=pred_prob)
    df = df.assign(prediction=choices)
    choices = np.random.choice([0, 1], size=n_rows, p=pred_prob)
    df = df.assign(trip_completed=choices)
    return df

In [60]:
orig_df = pd.read_parquet(ORIG_DATA_PATH, engine='fastparquet')
orig_df = format_data_df(orig_df, [0.2, 0.8])
orig_df

,datetime,driver_id,conv_rate,acc_rate,avg_daily_trips,created,prediction,trip_completed
0,2021-07-13 11:00:00+00:00,1005,0.373837,0.154890,498,2021-07-28 11:08:04.802,1,1
1,2021-07-13 12:00:00+00:00,1005,0.571627,0.643958,656,2021-07-28 11:08:04.802,0,1
2,2021-07-13 13:00:00+00:00,1005,0.399909,0.993888,722,2021-07-28 11:08:04.802,0,1
3,2021-07-13 14:00:00+00:00,1005,0.967468,0.788458,424,2021-07-28 11:08:04.802,1,1
4,2021-07-13 15:00:00+00:00,1005,0.024679,0.956064,569,2021-07-28 11:08:04.802,1,0
...,...,...,...,...,...,...,...,...
1802,2021-07-28 09:00:00+00:00,1001,0.089418,0.311234,485,2021-07-28 11:08:04.802,1,1
1803,2021-07-28 10:00:00+00:00,1001,0.222534,0.927691,114,2021-07-28 11:08:04.802,1,1
1804,2021-04-12 07:00:00+00:00,1001,0.175219,0.761434,385,2021-07-28 11:08:04.802,1,1
902,2021-07-20 23:00:00+00:00,1003,0.025968,0.109748,55,2021-07-28 11:08:04.802,1,1


In [61]:
mock_df = pd.read_parquet(DRIFT_DATA_PATH, engine='fastparquet')
mock_df = format_data_df(mock_df, [0.2, 0.8])
mock_df

,datetime,driver_id,conv_rate,acc_rate,avg_daily_trips,created,prediction,trip_completed
0,2021-07-19 23:00:00+00:00,1003,0.562670,0.711753,817,2021-07-28 11:08:04.802,1,0
1,2021-07-18 06:00:00+00:00,1005,0.747795,0.729159,664,2021-07-28 11:08:04.802,1,1
2,2021-07-28 09:00:00+00:00,1003,0.577423,0.600396,800,2021-07-28 11:08:04.802,1,0
3,2021-07-27 10:00:00+00:00,1002,0.676030,0.587644,820,2021-07-28 11:08:04.802,0,1
4,2021-07-23 05:00:00+00:00,1001,0.867539,0.571839,754,2021-07-28 11:08:04.802,1,1
...,...,...,...,...,...,...,...,...
95,2021-07-20 09:00:00+00:00,1004,0.784332,0.550629,741,2021-07-28 11:08:04.802,0,1
96,2021-07-23 14:00:00+00:00,1001,0.682082,0.600372,752,2021-07-28 11:08:04.802,1,1
97,2021-07-24 12:00:00+00:00,1004,0.732227,0.874406,841,2021-07-28 11:08:04.802,1,1
98,2021-07-27 17:00:00+00:00,1003,0.768284,0.835585,769,2021-07-28 11:08:04.802,1,1


In [62]:
orig_df.describe()

,driver_id,conv_rate,acc_rate,avg_daily_trips,prediction,trip_completed
count,1807.000000,1807.000000,1807.000000,1807.000000,1807.000000,1807.000000
mean,1003.000000,0.488267,0.505205,500.871057,0.794134,0.785833
std,1.413822,0.291862,0.291230,293.412315,0.404445,0.410357
min,1001.000000,0.000482,0.000542,0.000000,0.000000,0.000000
25%,1002.000000,0.238879,0.251682,236.000000,1.000000,1.000000
50%,1003.000000,0.491606,0.507843,506.000000,1.000000,1.000000
75%,1004.000000,0.732576,0.770225,754.000000,1.000000,1.000000
max,1005.000000,0.998767,0.999445,998.000000,1.000000,1.000000


In [63]:
mock_df.describe()

,driver_id,conv_rate,acc_rate,avg_daily_trips,prediction,trip_completed
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000
mean,1002.760000,0.731518,0.701609,751.450000,0.720000,0.85000
std,1.341791,0.090697,0.095636,78.185402,0.451261,0.35887
min,1001.000000,0.500000,0.500000,500.000000,0.000000,0.00000
25%,1001.750000,0.674897,0.627561,698.000000,0.000000,1.00000
50%,1003.000000,0.728371,0.688873,749.000000,1.000000,1.00000
75%,1004.000000,0.786183,0.771097,796.250000,1.000000,1.00000
max,1005.000000,0.995000,0.995000,994.000000,1.000000,1.00000


## Check data quality

In [64]:
import dataclasses
from typing import Dict, List, Optional

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.model_monitoring import ModelMonitoring
from evidently.model_monitoring import CatTargetDriftMonitor
from evidently.model_monitoring import ClassificationPerformanceMonitor
from evidently.model_monitoring import DataDriftMonitor
from evidently.model_monitoring import DataQualityMonitor
from evidently.model_monitoring import NumTargetDriftMonitor
from evidently.model_monitoring import ProbClassificationPerformanceMonitor
from evidently.model_monitoring import RegressionPerformanceMonitor

from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab, CatTargetDriftTab

from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection, CatTargetDriftProfileSection, ClassificationPerformanceProfileSection

@dataclasses.dataclass
class LoadedDataset:
    name: str
    references: pd.DataFrame
    monitors: List[str]
    column_mapping: ColumnMapping

EVIDENTLY_MONITORS_MAPPING = {
    "cat_target_drift": CatTargetDriftMonitor,
    "data_drift": DataDriftMonitor,
    "data_quality": DataQualityMonitor,
    "num_target_drift": NumTargetDriftMonitor,
    "regression_performance": RegressionPerformanceMonitor,
    "classification_performance": ClassificationPerformanceMonitor,
    "prob_classification_performance": ProbClassificationPerformanceMonitor,
}

In [65]:
DATETIME_COL = "datetime"
NUMERICAL_COLS = ["conv_rate", "acc_rate", "avg_daily_trips"]
CATEGORICAL_COLS = []
TARGET_COL = "trip_completed"
PREDICTION_COL = "prediction"
column_mapping = ColumnMapping(
    target=TARGET_COL,
    prediction=PREDICTION_COL,
    numerical_features=NUMERICAL_COLS,
    categorical_features=CATEGORICAL_COLS,
    datetime=DATETIME_COL
)
references = orig_df
current_data = mock_df

column_mapping

ColumnMapping(target='trip_completed', prediction='prediction', datetime='datetime', id=None, numerical_features=['conv_rate', 'acc_rate', 'avg_daily_trips'], categorical_features=[], datetime_features=None, target_names=None, task=None, pos_label=1)

In [67]:
monitors = ["data_drift", "classification_performance", "cat_target_drift"]
monitoring = ModelMonitoring(
    monitors=[EVIDENTLY_MONITORS_MAPPING[k]() for k in monitors],
    options=[],
)
monitoring.execute(references, current_data, column_mapping)
for metric, value, labels in monitoring.metrics():
    report = f"{metric.name} | {value} | {labels}"
    print(report)

data_drift:share_drifted_features | 0.6 | None
data_drift:n_drifted_features | 3 | None
data_drift:dataset_drift | True | None
data_drift:p_value | 0.05891373136841793 | {'feature': 'trip_completed', 'feature_type': 'cat'}
data_drift:p_value | 0.06121044396990028 | {'feature': 'prediction', 'feature_type': 'cat'}
data_drift:p_value | 0.7746270563557061 | {'feature': 'acc_rate', 'feature_type': 'num'}
data_drift:p_value | 0.9090495641010653 | {'feature': 'avg_daily_trips', 'feature_type': 'num'}
data_drift:p_value | 0.8911643440142998 | {'feature': 'conv_rate', 'feature_type': 'num'}
classification_performance:quality | 0.6651909241837299 | {'dataset': 'reference', 'metric': 'accuracy'}
classification_performance:quality | 0.49548068637368403 | {'dataset': 'reference', 'metric': 'precision'}
classification_performance:quality | 0.4956099646977472 | {'dataset': 'reference', 'metric': 'recall'}
classification_performance:quality | 0.49549480951293184 | {'dataset': 'reference', 'metric': '

In [68]:
bcancer_data_and_target_drift_dashboard = Dashboard(tabs=[DataDriftTab(verbose_level=0), CatTargetDriftTab(verbose_level=0)])
bcancer_data_and_target_drift_dashboard.calculate(references, current_data, column_mapping=column_mapping)
# bcancer_data_and_target_drift_dashboard.show()
# bcancer_data_and_target_drift_dashboard.save('data_and_target_drift.html')

In [69]:
bcancer_target_and_data_drift_profile = Profile(sections=[DataDriftProfileSection(), CatTargetDriftProfileSection()])
bcancer_target_and_data_drift_profile.calculate(references, current_data, column_mapping=column_mapping)
bcancer_target_and_data_drift_profile.json()

'{"data_drift": {"name": "data_drift", "datetime": "2022-10-02 23:45:49.651345", "data": {"utility_columns": {"date": "datetime", "id": null, "target": "trip_completed", "prediction": "prediction"}, "cat_feature_names": ["trip_completed", "prediction"], "num_feature_names": ["acc_rate", "avg_daily_trips", "conv_rate"], "datetime_feature_names": ["created"], "target_names": null, "options": {"confidence": null, "drift_share": 0.5, "nbinsx": 10, "xbins": null}, "metrics": {"n_features": 5, "n_drifted_features": 3, "share_drifted_features": 0.6, "dataset_drift": true, "acc_rate": {"current_small_hist": [[0.6060606060606062, 1.8181818181818188, 4.848484848484839, 3.030303030303038, 3.0303030303030245, 4.646464646464648, 1.0101010101010104, 0.6060606060606062, 0.20202020202020207, 0.40404040404040414], [0.5, 0.5495, 0.599, 0.6485000000000001, 0.698, 0.7475, 0.797, 0.8465, 0.896, 0.9455, 0.995]], "ref_small_hist": [[0.9639796949027413, 1.0027604872264149, 1.0526215059282804, 0.94735935533545

In [59]:
bcancer_target_and_data_drift_profile = Profile(sections=[ClassificationPerformanceProfileSection()])
bcancer_target_and_data_drift_profile.calculate(current_data, current_data, column_mapping=column_mapping)
bcancer_target_and_data_drift_profile.json()

'{"classification_performance": {"name": "classification_performance", "datetime": "2022-10-02 23:44:02.230209", "data": {"utility_columns": {"date": "datetime", "id": null, "target": "trip_completed", "prediction": "prediction"}, "cat_feature_names": [], "num_feature_names": ["acc_rate", "avg_daily_trips", "conv_rate"], "datetime_feature_names": ["created"], "target_names": null, "metrics": {"reference": {"accuracy": 0.63, "precision": 0.4255952380952381, "recall": 0.44666666666666666, "f1": 0.43242828654701637, "metrics_matrix": {"0": {"precision": 0.125, "recall": 0.08, "f1-score": 0.09756097560975609, "support": 25}, "1": {"precision": 0.7261904761904762, "recall": 0.8133333333333334, "f1-score": 0.7672955974842767, "support": 75}, "accuracy": 0.63, "macro avg": {"precision": 0.4255952380952381, "recall": 0.44666666666666666, "f1-score": 0.43242828654701637, "support": 100}, "weighted avg": {"precision": 0.5758928571428572, "recall": 0.63, "f1-score": 0.5998619420156465, "support":